In [2]:
from IPython.display import Code

# Example: Compare MIPS of RISC-V Instruction Set Simulators

Typically MLonMCU would be used to benchmark TinyML workloads on real wardware or simulators. However it's flexibility also allows some interesting experiments not directly related to Embedded ML. In the following it the performance of some RISC-V ISA Simulators is compared using the MLonMCU command line or Python API.

## Supported components

**Models:** Any (`sine_model` used below)

**Frontends:** Any (`tflite` used below)

**Frameworks/Backends:** Any (`tvmaotplus` used below)

**Platforms/Targets:** `etiss`, `spike`, `ovpsim` (`etiss` and `spike` used below)

## Prerequisites

If not done already, setup a virtual python environment and install the required packages into it. (See `requirements.txt`)

In [4]:
Code(filename="requirements.txt")

-r ../../requirements.txt

A custom `environment.yml` template is used here. Make sure that `allow_extensions: true` is set!

In [6]:
Code(filename="environment.yml.j2")

---
home: "{{ home_dir }}"
logging:
  level: INFO
  to_file: false
  rotate: false
cleanup:
  auto: true
  keep: 5
paths:
  deps: deps
  logs: logs
  results: results
  plugins: plugins  # Store plugins in environment dir!
  temp: temp
  models:
    - "{{ home_dir }}/models"
repos:
  abc:
    url: https://github.com/githubtraining/hellogitworld.git
    ref: master
  tensorflow:
    url: "https://github.com/tensorflow/tflite-micro.git"
    ref: f868a900f1f788fd0a55b5655b47fc352f9a0239
  mlif:
    url: "https://github.com/tum-ei-eda/mlonmcu-sw.git"
    ref: 3f8e66b956fe1cf0d85a18d39d684816e92e2751
frameworks:
  default: tflm
  tflm:
    enabled: true
    backends:
      default: tflmi
      tflmc:
        enabled: false
      tflmi:
        enabled: true
        features:
          debug_arena: true
    features:
      muriscvnn: false
      cmsisnn: false
frontends:
  tflite:
    enabled: true
    features: []
toolchains:
  gcc: true
  llvm: false
platforms:
  mlif:
    enabled: true
    features:
      debug: true
      validate: true
targets:
  default: host_x86
  abc:
    enabled: true
    features: []
  host_x86:
    enabled: true
    features:
      gdbserver: true
postprocesses:
  use:
  # - detailed_cycles
  # - average_cycles
  # - filter_cols
  # - features2cols
  # - config2cols
  # - bytes2kb
  # - visualize
vars:
  allow_extensions: true
  llvm.version: "11.1.0"
  llvm.distribution: "x86_64-linux-gnu-ubuntu-16.04"
  runs_per_stage: true
  riscv_gcc.dl_url: "https://syncandshare.lrz.de/dl/fiWBtDLWz17RBc1Yd4VDW7/GCC/default/2023.11.27/Ubuntu/20.04/rv32gc_ilp32d.tar.xz"
flags: {}

Do not forget to set your `MLONMCU_HOME` environment variable first if not using the default location!

## Extending MLonMCU with Plugins

After running `mlonmcu init ...` and before executing `mlonmcu setup ...` you will need to populate the `plugins` directory of the environment. Alternatively `~/.config/mlonmcu/plugins` can be used to define extensions.

### Overview

A minimal plugin only consists of the single file `extensions.py`, the entry point which is called automatically by MLonMCU if `allow_extensions: true` is set in the environment. The name of this file is essential and can not be changed. If multiple files are used to build more complex plugins, the following workaround is required to allow relative imports:

```
import os
import sys
sys.path.insert(0, os.path.dirname(__file__))
```

### Example Extension: ABC Target

Here a simple example of a dummy (non-functional) target called `abc` is discussed. This involves setting up dependencies (clone repo), providing custom (out-of-tree) CMake scripts for compilation and registering a new `ABCTarget` to MLonMCU.

This is the file structure of the `plugins` directory:

```
plugins
├── extensions.py
├── __init__.py
├── _setup
│   ├── __init__.py
│   └── tasks
│       ├── abc.py
│       └── __init__.py
└── _target
    └── riscv
        └── abc.py
```

Everytime when launching MLonMCU the line `WARNING - Executing Plugin: mlonmcu.extensions.environment` will; be logged:

In [11]:
!python3 -m mlonmcu.cli.main models

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - Loading extensions.py (Environment)
spec ModuleSpec(name='mlonmcu.extensions.environment', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f0e809a3e50>, origin='/work/git/mlonmcu/mlonmcu/workspace_minimal/plugins/extensions.py')
mod <module 'mlonmcu.extensions.environment' from '/work/git/mlonmcu/mlonmcu/workspace_minimal/plugins/extensions.py'>
WARNING - Excuting Plugin: mlonmcu.extensions.environment
validates2 {'install_corstone300': <function _validate_corstone300 at 0x7f0e80a67550>, 'install_arm_gcc': <function _validate_corstone300 at 0x7f0e80a67550>, 'clone_cmsisnn': <function _validate_cmsisnn at 0x7f0e80a67940>, 'clone_cmsis': <function _validate_cmsis at 0x7f0e80a67af0>, 'clone_ethosu_platform': <function _validate_ethosu_platform at 0x7f0e80a67c10>, 'clone_espidf': <function _validate_espidf at 0x7f0e80a6c0d0>, 'install_espidf': <function _validate_espidf at 0x7f0e80a6c0d